In [2]:
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import string
import re
from nltk.corpus import stopwords
import os

In [5]:
os.listdir('enron1/spam/')

['4743.2005-06-25.GP.spam.txt',
 '1309.2004-06-08.GP.spam.txt',
 '0726.2004-03-26.GP.spam.txt',
 '0202.2004-01-13.GP.spam.txt',
 '3988.2005-03-06.GP.spam.txt',
 '0758.2004-04-02.GP.spam.txt',
 '5145.2005-09-04.GP.spam.txt',
 '4382.2005-04-26.GP.spam.txt',
 '2794.2004-11-11.GP.spam.txt',
 '1489.2004-07-03.GP.spam.txt',
 '0391.2004-02-08.GP.spam.txt',
 '3312.2004-12-27.GP.spam.txt',
 '4735.2005-06-20.GP.spam.txt',
 '2075.2004-09-04.GP.spam.txt',
 '1993.2004-08-26.GP.spam.txt',
 '1313.2004-06-09.GP.spam.txt',
 '4577.2005-05-27.GP.spam.txt',
 '3714.2005-02-05.GP.spam.txt',
 '1180.2004-05-21.GP.spam.txt',
 '0256.2004-01-20.GP.spam.txt',
 '5132.2005-09-02.GP.spam.txt',
 '3383.2005-01-04.GP.spam.txt',
 '5020.2005-08-16.GP.spam.txt',
 '1656.2004-07-19.GP.spam.txt',
 '3042.2004-12-05.GP.spam.txt',
 '0555.2004-02-22.GP.spam.txt',
 '5155.2005-09-05.GP.spam.txt',
 '1107.2004-05-15.GP.spam.txt',
 '3591.2005-01-26.GP.spam.txt',
 '2649.2004-10-27.GP.spam.txt',
 '0407.2004-02-11.GP.spam.txt',
 '3173.2

In [26]:
def load_files(directory):
    result = []
    for fname in os.listdir(directory):
        #print(fname)
        with open(directory+"/"+fname,'r',encoding='ISO-8859-1') as f:
            result.append(f.read())
    return result

In [30]:
positives = load_files('enron1/spam/')
negatives = load_files('enron1/ham/')

In [58]:
len(positives),len(negatives)

(1500, 3672)

In [59]:
positives[0]

"Subject: what up , , your cam babe\nwhat are you looking for ?\nif your looking for a companion for friendship , love , a date , or just good ole '\nfashioned * * * * * * , then try our brand new site ; it was developed and created\nto help anyone find what they ' re looking for . a quick bio form and you ' re\non the road to satisfaction in every sense of the word . . . . no matter what\nthat may be !\ntry it out and youll be amazed .\nhave a terrific time this evening\ncopy and pa ste the add . ress you see on the line below into your browser to come to the site .\nhttp : / / www . meganbang . biz / bld / acc /\nno more plz\nhttp : / / www . naturalgolden . com / retract /\ncounterattack aitken step preemptive shoehorn scaup . electrocardiograph movie honeycomb . monster war brandywine pietism byrne catatonia . encomia lookup intervenor skeleton turn catfish .\n"

In [48]:
from nltk import PorterStemmer, SnowballStemmer 
from nltk.stem import WordNetLemmatizer

In [62]:
def preprocess_sentence(sentence):
    
    #-- 
    # Tokenize
    #---
    words = word_tokenize(sentence)
    #print(words)
    stpwords = stopwords.words("english")
    words_processed = []
    for w in words:
        if w not in stpwords:
            if w not in string.punctuation:
                words_processed.append(w)
    #print("w",words_processed)
    wordnetLemmatizer = WordNetLemmatizer()           
    keywords=[w for w in words_processed if w in wordnetLemmatizer.lemmatize(w)]
    return keywords
    

positive_examples = [preprocess_sentence(email) for email in positives]
negative_examples = [preprocess_sentence(email) for email in negatives]

In [66]:
positive_examples = [(email, 1) for email in positive_examples]
negative_examples = [(email, 0) for email in negative_examples]

In [70]:
import random
all_examples = positive_examples + negative_examples
random.shuffle(all_examples)

In [72]:
print('{} emails processed.'.format(len(all_examples)))

5172 emails processed.


In [83]:
import collections 
def feature_extraction(tokens):
    #print('tokens',tokens)
    counts = collections.Counter(tokens)
    return dict(counts)

featurized = [(feature_extraction(email_token),label) for email_token,label in all_examples]

In [88]:
def train_test_split(dataset, train_size=0.8):
    num_training_examples = int(len(dataset) * train_size)
    return dataset[:num_training_examples], dataset[num_training_examples:]

training_set, test_set = train_test_split(featurized, train_size=0.7)

In [89]:
import nltk
model = nltk.classify.NaiveBayesClassifier.train(training_set)

In [91]:
training_error = nltk.classify.accuracy(model, training_set)
print('Model training complete. Accuracy on training set: {}'.format(
    training_error))

Model training complete. Accuracy on training set: 0.961878453038674


In [92]:
testing_error = nltk.classify.accuracy(model, test_set)
print('Accuracy on test set: {}'.format(testing_error))

Accuracy on test set: 0.9561855670103093
